In [1]:
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o", "gpt4-turbo"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'master_key': os.getenv('ACCOUNT_KEY'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER_EMPLOYEE'),
    'search_endpoint': os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT'),
    'serch_key': os.getenv('AZURE_SEARCH_KEY'),
    'search_api_version': os.getenv('AZURE_SEARCH_API_VERSION')
}


for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
credential = DefaultAzureCredential()
from azure.identity import AzureCliCredential

endpoint = settings['search_endpoint']

credential = AzureCliCredential()
token = credential.get_token("https://cognitiveservices.azure.com/.default")

print("TOKEN", token.token)

In [3]:
client = SearchClient(endpoint=endpoint, index_name="proc-agent", credential=credential)

In [4]:
onboarding_buddy_prompt = """
You are an experienced onboarding buddy, working with a new hire. You need to guide the user through the onboarding process. You have all tthe information you need to help the user onboard. You are responsible for ensuring the user is onboarded successfully.
If you do not know the answer to a question, you can ask the supervisor for help. You can retrieve the employee information from the database based on the Employee ID, and retrieve information from the knowledge base by providing the search terms to the function "search".  
"""

supervisor_prompt = """
You are a supervisor. You are responsible for ensuring that the onboarding buddy is able to guide the new hire through the onboarding process. You need to provide the onboarding buddy with the information they need to help the new hire onboard successfuly.
You are responsible for the onboarding process and ensuring that the new hire is onboarded successfully. 
"""

In [5]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="The new hire.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

onboarding_buddy = autogen.ConversableAgent(
    name="onboarding_buddy",
    system_message= onboarding_buddy_prompt,
    llm_config=llm_config,
)
supervisor = autogen.ConversableAgent(
    name="supervisor",
    system_message=supervisor_prompt,
    llm_config=llm_config,
)

temp_dir = tempfile.TemporaryDirectory()

# Create a Docker command line code executor.
executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",  # Execute code using the given docker image name.
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration that uses docker.
code_executor_agent_using_docker = ConversableAgent(
    "code_executor_agent_docker",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the docker command line code executor.
    human_input_mode="NEVER",  # Always take human input for this agent for safety.
)

groupchat = autogen.GroupChat(agents=[user_proxy, onboarding_buddy, supervisor, code_executor_agent_using_docker], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [6]:
import json
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

# Initialize the Cosmos client
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY})
# Get the database and container
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)

@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="get_employee_info", description="Retrieve employee data from the database, including the start date and mandatory exam asignments.")
def get_employee_info(EmployeeId: str) -> str:
    try:
        # Query the container for the customer with the specified ID
        query = f"SELECT * FROM c WHERE c.EmployeeId = '{EmployeeId}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            # If the customer is found, return the customer information as JSON
            employee_info = items[0]
            return json.dumps(employee_info, default=str)
        else:
            # If the customer is not found, return an error message as JSON
            error_message = {'error': f"Employee with ID '{EmployeeId}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        # If there's an error querying the container, return the error message as JSON
        error_message = {'error': str(e)}
        return json.dumps(error_message)

In [7]:
@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="search", description="Search the knowledge base for documentation that may help the user.")
def search(query: str):
    payload = json.dumps(
        {
            "search": query,
            "vectorQueries": [{"kind": "text", "text": query, "k": 5, "fields": "content"}],
            "queryType": "semantic",
            "captions": "extractive",
            "answers": "extractive|count-3",
            "queryLanguage": "en-US",
        }
    )

    response = list(client.search(payload))

    output = []
    for result in response:
        result.pop("titleVector")
        result.pop("contentVector")
        output.append(result)

    return output

The return type of the function 'search' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


In [8]:
user_proxy.initiate_chat(
    manager, message="Hi! I'm a new employee."
)

User_proxy (to chat_manager):

Hi! I'm a new employee.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
onboarding_buddy (to chat_manager):

Welcome to the team! I'm here to help you with the onboarding process. To get started, I'll need your Employee ID to retrieve your relevant information and make sure everything is set up correctly for you.

Could you please provide me with your Employee ID?

--------------------------------------------------------------------------------
User_proxy (to chat_manager):

123

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
onboarding_buddy (to chat_manager):

***** Suggested tool call (call_XYR14RDhDKk3moiKJ9nbJAM7): get_employee_info *****
Arguments: 
{"EmployeeId":"123"}
**********************************************************************************

------------------------------------------------------------------------

KeyboardInterrupt: 